## Created Comprehensive Index of open datasets about Lima 

***Google Drive Link to Google Sheets "Economy & Finance Datasets link":***

https://docs.google.com/spreadsheets/d/1zgVZjVG8Ih3EMresJsD2Fn8iMjGPLykMOwfEqMM_DJ4/edit?usp=drive_link

***Google Drive Link to Google Sheets "Datasets Related to Lima":***

https://docs.google.com/spreadsheets/d/1ypGHYG7-MxYOWPTxgKpDwhcuVHJcactaoSNzUcnqPAQ/edit?usp=drive_link

In [46]:
# !pip install gspread
# !pip install google-auth google-auth-oauthlib google-auth-httplib2
# !pip install translate
# !pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 55.1/55.1 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB ? eta 0:00:00
     -------------------------------------- 133.4/133.4 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 58.8/58.8 kB 622.7 kB/s eta 0:00:00
     ---------------------------------------- 65.0/65.0 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 53.6/53.6 kB 2.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17461 sha256=240c105cf6db82405993c29bc3b5635ffd508740fcb5c4cfe4befbeaaa5f725c
  Stored in directory: c:\users\l\appdata\local\pip\cache\wheels\54\ca\27\562b6eac3a495887e4b44bac3a1efe925fa603d085ba89a21d
Successfully built googletrans
  Attempting uninstall: char

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.


# Below is Python Code to web scrape data from website and store in Google Sheets

In [127]:

import requests
from bs4 import BeautifulSoup
import re
import gspread
from google.auth import exceptions
from google.oauth2 import service_account
from googletrans import Translator

def scrape_website(base_url, start_page, end_page):
    all_dataset_details = []
    
    for page in range(start_page, end_page + 1):
        url = f'{base_url}&page=0%2C{page}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # Extracting dataset details
        dataset_details = []

        # Find all article elements with class 'node-search-result'
        datasets = soup.find_all('article', class_='node-search-result')

        for index, dataset in enumerate(datasets, start=1):
            dataset_name = dataset.find('h2').text.strip()
            
            # Find the 'h2' tag within the specified div
            dataset_div = soup.find('div', class_='col-md-10 col-lg-11 col-xs-10 search-result search-result-dataset')
            h2_tag = dataset_div.find('h2')

            # Find the 'a' tag within the 'h2' tag
            dataset_link = h2_tag.find('a')

            # Extract the relative URL
            relative_url = dataset_link['href'] if dataset_link else None

            # Constructing the full dataset URL
            dataset_url = relative_url if relative_url.startswith('http') else f'https://www.datosabiertos.gob.pe{relative_url}'

            
            # Extract category
            category_element = dataset.find('a', class_='name', href=re.compile(r'/field_topic/'))
            category = category_element.text.strip() if category_element else None

            # Extract sub-category
            sub_category_element = dataset.find('div', class_='group-membership')
            sub_category = sub_category_element.text.strip() if sub_category_element else None

            # Extract short description
            description_element = dataset.find('div', class_='node-description')
            short_description = description_element.text.strip() if description_element else None

            # Translate to English
            translator = Translator()
            dataset_name_en = translator.translate(dataset_name, dest='en').text
            category_en = translator.translate(category, dest='en').text if category else None
            sub_category_en = translator.translate(sub_category, dest='en').text if sub_category else None
            short_description_en = translator.translate(short_description, dest='en').text if short_description else None

            file_availability = any(format in dataset.text for format in ['pdf', 'csv', 'zip', 'url','docx','xlsx'])

            # Check if 'data-and-resources' div is present
            resources_div = dataset.find('div', class_='data-and-resources')
            observation = resources_div.text.strip() if resources_div else None

            dataset_details.append({
                'Index': index,
                'Dataset Name': dataset_name_en,
                'Dataset URL': dataset_url,
                'Category': category_en,
                'Sub-category': sub_category_en,
                'Short Description': short_description_en,
                'File Availability': file_availability,
                'Observation': observation
            })
        all_dataset_details.extend(dataset_details)

    return all_dataset_details

# Function to authenticate with Google Sheets API
def authenticate_google_sheets(credential_path, spreadsheet_url):
    scope = ['https://www.googleapis.com/auth/spreadsheets']
    credentials = service_account.Credentials.from_service_account_file(credential_path, scopes=scope)
    gc = gspread.authorize(credentials)
    spreadsheet = gc.open_by_url(spreadsheet_url)
    sheet = spreadsheet.sheet1
    return sheet

# Function to write data to Google Sheet
def write_to_google_sheet(sheet, data):
    # Get the current number of rows in the sheet
    current_rows = len(sheet.get_all_values())

    # Write headers only if the sheet is empty
    if current_rows == 0:
        headers = list(data[0].keys())
        sheet.insert_row(headers, 1)

    # Write data starting from the next available row
    start_row = current_rows + 1
    for row_data in data:
        row_values = list(row_data.values())
        sheet.insert_row(row_values, start_row)
        start_row += 1


if __name__ == '__main__':
    base_url = 'https://www.datosabiertos.gob.pe/search/field_topic/econom%C3%ADa-y-finanzas-29?query=&sort_by=changed&sort_order=DESC'
    start_page = 0
    end_page = 119
    
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1zgVZjVG8Ih3EMresJsD2Fn8iMjGPLykMOwfEqMM_DJ4/edit#gid=0'
    
    credential_path = 'credentials.json'
    # Scrape the website
    dataset_details = scrape_website(base_url, start_page, end_page)
    
    # Authenticate with Google Sheets
    sheet = authenticate_google_sheets(credential_path, spreadsheet_url)
    
    # Write data to Google Sheet
    write_to_google_sheet(sheet, dataset_details)


C:\Users\l\AppData\Local\Temp\ipykernel_25768\1585325264.py:122: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  sheet = authenticate_google_sheets(credential_path, spreadsheet_url)


<h2> **Note: The above code might result in API error** </h2>

The error you're encountering indicates that you've exceeded the quota limit for the number of write requests to the Google Sheets API. The default limit for the "Write requests per minute per user" quota is 60 requests.

To resolve this issue, you have a few options:

    - Wait for the Quota to Reset:
    The quota is likely to reset after a certain period. You can wait for some time and then try running your script again.

    - Check and Increase Quota:
    You can check your current quota usage and request a quota increase if needed. Visit the Google Cloud Console Quotas page, select your project, and look for the "Write requests per minute per user" quota. If needed, you can submit a request for a higher quota limit.

    - Optimize Your Code:
    If your script is making a large number of unnecessary requests, consider optimizing it to reduce the number of API calls. For example, you might want to review whether you need to clear the sheet and rewrite all data on every run. You could also implement caching strategies to avoid unnecessary API calls.

    Remember that if you're using the Google Sheets API frequently, especially during development and testing, you might hit these limits.  
    
<br>    
    
**PS: I Tried to run three pages eachtime**     

## Below are some important datasets that can be used for analysis to extract actionable insights in order to develop the Economy of Lima 

1) **Santa Anita District Municipality** https://www.datosabiertos.gob.pe/group/municipalidad-distrital-de-santa-anita \
1.1) Revenue Collected from the District Municipality of Santa Anita 2023
https://www.datosabiertos.gob.pe/dataset/ingresos-recaudados-de-la-municipalidad-distrital-de-santa-anita/resource/a29823c5-035a-494d

    Data on Tax Collection Income of the District Municipality of Santa Anita corresponding to the year 2023 (January to May).The types of income correspond to Property Tax, Excise Taxes (Public Cleaning, Parks and Gardens or Serenazgo) and other concepts such as: forms, cards, certificates, fines, etc.This information is important to contribute to the transparency of municipal management and the openness of data.

    This dataset is characterized by:
    Ubigeo:  Geographic location code where the payment was made.
    Department:  Department where the Payment was made.
    Province:  Province where the Payment was made.
    District:  District where the Payment was made.
    Citizen Data:  Id that identifies the taxpayer (anonymized)
    Type of Tax : Type of Tax collected, in this case property tax.
    Years paid:  The years that remained to be paid to the citizen.
    Number of properties:  Number of paid real estate possessions.
    Property Location:  Reference location of the property.
    Tax:  Tax paid.
    Default interest:  Interest for late payment by the taxpayer.
    Balance:  Balance that remains to be paid.
    Date:  Date of payment.
    Receipt:  Receipt number.
    Payment:  Total paid per row. Expressed in Peruvian Sol currency.
    

    #### Type of Analysis can be done :    
    `Understand Tax Contribution, Identify Taxpayer Behavior, Property Ownership and Tax Payments, Financial Health of Citizens, Temporal Trends, Geographic Distribution, Correlation with Local Development, Forecasting and Planning, Policy Implications`    

2) **District Municipality of ATE** https://www.datosabiertos.gob.pe/group/municipalidad-distrital-de-ate  
    
    2.1) FUEL EXPENDITURE IN VEHICLE UNITS OF THE DISTRICT MUNICIPALITY OF ATE – [MDA]
    https://www.datosabiertos.gob.pe/dataset/gasto-de-combustible-en-unidades-vehiculares-de-la-municipalidad-distrital-de-ate-%E2%80%93-mda

    Data on fuel expenses in vehicle units of the District Municipality of Ate (MDA) used by the entity to carry out its daily activities. The MDA, as a local government entity, uses a fleet of vehicles to perform various tasks, such as transporting personnel, waste collection, infrastructure maintenance, among others.

    2.2) INCOME FROM ARBITRATION FEES COLLECTED BY CASH OFFICE OF THE DISTRICT MUNICIPALITY OF ATE FROM THE YEAR 2019 TO 2022 – [MDA]
    https://www.datosabiertos.gob.pe/dataset/ingresos-por-tasas-de-arbitrios-recaudado-por-caja-de-la-municipalidad-distrital-de-ate-del

    Data from the total income generated through municipal taxes payments made in the treasury of the aforementioned municipality during that four-year period. Municipal taxes are local taxes that are charged to taxpayers for the provision of public services and the maintenance of infrastructure in the Ate district. These taxes may include, for example, the collection of solid waste, the maintenance of green areas, the cleaning of public roads, among others.
    The analysis of these income can provide relevant information about the financial situation of the District Municipality of Ate, its capacity to provide public services and its management in terms of municipal tax collection. It can also be useful to evaluate the efficiency of collection policies and to make future income projections.


    #### Type of Analysis can be done :
    - Fuel Expenditure (Gasto de Combustible)\
    `Operational Efficiency, Cost Management`
    -  Income from Arbitration Fees (Ingresos por Tasas de Arbitrios) \
    `Financial Health Assessment, Service Provision Capability, Efficiency of Tax Collection, Future Projections`

    - Overall Insights for Lima's Economy: \
    `Local Government Efficiency, Local Government Efficiency, Environmental Sustainability, Community Impact`

3) **District Municipality of Jesús María - Lima**
https://www.datosabiertos.gob.pe/group/municipalidad-distrital-de-jes%C3%BAs-mar%C3%ADa-lima 
    
    3.1) MDJM - List of Purchase Orders and Services (2nd Sem. 2021)
    https://www.datosabiertos.gob.pe/dataset/mdjm-lista-de-ordenes-de-compra-y-servicios-2do-sem-2021

    3.2) MDJM - List of Purchase Orders and Services (1st Sem. 2022)
    https://www.datosabiertos.gob.pe/dataset/mdjm-lista-de-ordenes-de-compra-y-servicios-1er-sem-2022

    List of Purchase Orders and Services of the District Municipality of Jesús María.
    Purchase Orders :
    A purchase order (also known as a purchase order or order note) is a  document issued by the buyer to request goods from the seller . It is very common to send orders to suppliers. From a customer's purchase order you can make the corresponding invoice more quickly.
    Service Orders :
    The service order is a  document that serves to record all information related to service requests . In this way, all requests are formalized within a certain standard to facilitate the tracking of orders, whether internal (employees) or external (customers).

    This Dataset is characterized by: Order Type, RUC Order, SIAF Purchase Order, Purchase Order Date, Amount, Supplier Name, Description, Purchase Date, Purchase Year, Purchase Month, Department, Province, District.     


    3.3) Personnel expenses by labor regime of the District Municipality of Jesús María – [MJM]
    https://www.datosabiertos.gob.pe/dataset/gastos-en-personal-por-r%C3%A9gimen-laboral-de-la-municipalidad-distrital-de-jes%C3%BAs-mar%C3%ADa-%E2%80%93-mjm
    Record of personnel expenses by employment regime in the District Municipality of Jesús María.


    3.4) Income from operating license District Municipality of Jesús María - [MJM]
    https://www.datosabiertos.gob.pe/dataset/ingresos-por-licencia-de-funcionamiento-municipalidad-distrital-de-jes%C3%BAs-mar%C3%ADa-mjm

    Registration of income from operating licenses in the District Municipality of Jesús María.

    3.5) Income from commercial properties of the District Municipality of Jesús María – [MJM]
    https://www.datosabiertos.gob.pe/dataset/ingresos-por-predios-comerciales-de-la-municipalidad-distrital-de-jes%C3%BAs-mar%C3%ADa-%E2%80%93-mjm

    Record of income from payments of commercial properties in the District Municipality of Jesús María. 


    #### Type of Analysis can be done :

    - MDJM - List of Purchase Orders and Services \
    `Procurement Efficiency, Supplier Relationships, Budget Allocation`
    - Personnel Expenses by Labor Regime \
    `Labor Cost Management, Workforce Dynamics`
    -  Income from Operating License \
    `Business Activity Assessment`
    - Income from Commercial Properties \
    `Property Revenue Trends, Impact on Local Economy`

    - Overall Insights for Lima's Economy \
    `Financial Health, Economic Drivers, Policy Recommendations`

4) **District Municipality of Pucusana**
https://www.datosabiertos.gob.pe/SEARCH/field_topic/econom%C3%ADa-y-finanzas-29/type/dataset?query=&sort_by=changed&sort_order=DESC&page=0%2C40 \

    4.1) Income collected from the District Municipality of Pucusana
    https://www.datosabiertos.gob.pe/dataset/ingresos-recaudados-de-la-municipalidad-distrital-de-pucusana

    The information corresponds to all  income collected  by the District Municipality of Pucusana in the year  2022  (January - September). The types of income correspond to property tax, taxes (public cleaning, parks and gardens or serenazgo) and other concepts such as: forms, cards, certificates, fines, etc.
    The location of this dataset is the Department of Lima, Province of Lima, District of Pucusana with Ubigeo 140118.
    Within this dataset you will find the movement number, type of movement, and the person's code. Personal data such as documents, names, surnames and address were correctly anonymized in compliance with current regulations. The amounts collected include the type of payment, property location, concept, cancellation date, among other data.
    Open data is important to contribute to the transparency of municipal management and the openness of data. 
     
     #### Type of Analysis can be done :
    `Revenue Composition, Temporal Trends, Geographic Analysis, Payment Types and Trends, Anomaly Detection`

5) **District Municipality of Miraflores**
https://www.datosabiertos.gob.pe/group/municipalidad-distrital-de-miraflores \
    
    5.1) Execution of investment projects of the District Municipality of Miraflores 2022- [Municipality of Miraflores]
    https://www.datosabiertos.gob.pe/dataset/ejecuci%C3%B3n-de-proyectos-de-inversi%C3%B3n-de-la-municipalidad-distrital-de-miraflores

    It is any expenditure of resources of public origin intended to create, increase, improve or replace the stocks of physical capital in the public domain, with the aim of expanding the country's capacity for the provision of services and production of goods. Capacity of the entity to spend the budgetary and financial resources assigned for investment projects.    

    5.2) Detail by Generic and Expense Item - [Municipality of Miraflores]
    https://www.datosabiertos.gob.pe/dataset/detalle-por-gen%C3%A9rica-y-rubro-de-gastos-municipalidad-de-miraflores
    Detail by Generic and Execution Area of Expenses 2020
    
    https://www.datosabiertos.gob.pe/dataset/detalle-por-gen%C3%A9rica-y-rubro-de-gastos-municipalidad-de-miraflores    
    Detail by Generic and Execution Area of Expenses 2020 - 2022   

    5.3) PIA and PIM Income - [Municipality of Miraflores] 
    https://www.datosabiertos.gob.pe/dataset/ingresos-pia-y-pim-municipalidad-de-miraflores 

    PIA and PIM Income 2022 - 2021

    5.4) Detail by Specific and Expenditure Item - [Municipality of Miraflores]
    https://www.datosabiertos.gob.pe/dataset/detalle-por-espec%C3%ADfica-y-rubro-de-gasto-municipalidad-de-miraflores 

    Detail by Specific and Expenditure Execution Area 2020 - 2022    

    5.5) Tax revenues - [Municipality of Miraflores]
    https://www.datosabiertos.gob.pe/dataset/ingresos-tributarios-%C2%A0municipalidad-de-miraflores 

    Tax revenues 2019 - 2020

    5.6) Proof of Payment - [Municipality of Miraflores]
    https://www.datosabiertos.gob.pe/dataset/comprobante-de-pago-municipalidad-de-miraflores 

    Proof of Payment 2015 - 2020    
    
    
    #### Type of Analysis can be done :
    - Execution of investment projects  \
    `Investment Priorities, Budget Execution, Project Impact`
    -  Detail by Generic and Expense Item \
    `Analyze trends over time, Compare expenses to identify priorities`
    - PIA and PIM Income \
    `Compare PIA with PIM, Assess income projections and financial planning`
    - Detail by Specific and Expenditure Item \
    `Analyze spending patterns, Evaluate the effectiveness of spending`
    -  Tax Revenues \
    `Examine trends over time, Identify the main sources of tax revenue`
    - Proof of Payment \
    `Analyze payment patterns, frequency, and amounts, Identify any trends or anomalies in payment behavior`


6) **Municipality of San Isidro - Lima**
https://www.datosabiertos.gob.pe/group/municipalidad-de-san-isidro-lima 
    
    6.1) Administration: Orders of Goods and Services - Municipality of San Isidro
    https://www.datosabiertos.gob.pe/dataset/administraci%C3%B3n-ordenes-de-bienes-y-servicios-municipalidad-de-san-isidro

    Orders of Goods and Services 2015 - 2023

    6.2) Budget: Expenses and Income - Municipality of San Isidro
    https://www.datosabiertos.gob.pe/dataset/presupuesto-gastos-e-ingresos-municipalidad-de-san-isidro-0

    Expenses and Revenue 2015 - 2023    

    6.3) Income: Tax Collection - Municipality of San Isidro
    https://www.datosabiertos.gob.pe/dataset/rentas-recaudacion-de-tributos-municipalidad-de-san-isidro

    Tribute Collection and Collection by Payment 2018 - 2023    

    6.4) Administration: Annual Contracting Plan - Municipality of San Isidro
    https://www.datosabiertos.gob.pe/dataset/administraci%C3%B3n-plan-anual-de-contrataciones-municipalidad-de-san-isidro

    Selection Processes Published at SEACE 2021-2023 MSI
    Processes of the 2021-2023 MSI Annual Hiring Plan
    PAC 2021-2023 MODIFIED MSI
    PAC 2021-2023 INITIAL MSI

    6.5) Administration: Payment Receipts - Municipality of San Isidro
    https://www.datosabiertos.gob.pe/dataset/administraci%C3%B3n-comprobantes-de-pago-municipalidad-de-san-isidro

    Payment Receipts 2018 - 2023

    6.6) Investment Works and Projects - Municipality of San Isidro
    https://www.datosabiertos.gob.pe/dataset/obras-y-proyectos-de-inversion-municipalidad-de-san-isidro

    Works in Execution by PIP's MSI    
    Technical File Profile MSI 

    6.7) Building Licenses Authorization and Control - Municipality of San Isidro
    
    Operating Licenses MSI 
    Construction Compliance MSI 
    Building Licenses MSI 


    #### Type of Analysis can be done :
    - Administration: Orders of Goods and Services \
    `Analyze trends in the types and volume of goods and services procured, Evaluate vendor relationships and spending patterns, Identify key suppliers and areas`
    -  Budget: Expenses and Income \
    `Analyze trends in budget allocation and expenditures over time, Evaluate the sources and sustainability of income, Identify areas of high expenditure and potential cost-saving opportunities`
    -  Income: Tax Collection \
    `Examine trends in tax revenue and types of taxes, Evaluate the impact of tax policies on revenue generation`
    - Administration: Annual Contracting Plan \
    `Analyze the types of contracts planned and executed, Evaluate the alignment of contracts with municipal objectives, Identify key focus areas for contracting`
    - Administration: Payment Receipts \
    `Analyze payment patterns, frequency, and amounts, Identify areas for improvement in payment processing`
    - Investment Works and Projects \
    `Evaluate the progress and impact of investment projects, Analyze the allocation of funds to specific projects, Identify areas for optimizing project management and execution`
    -  Building Licenses Authorization and Control \
    `Evaluate the impact of construction activities on the local economy, Identify trends in urban development and construction activities`


7) **Regional Government of Lima**
https://www.datosabiertos.gob.pe/group/gobierno-regional-de-lima 

    7.1) Investment Projects of the Regional Government of Lima
    https://www.datosabiertos.gob.pe/dataset/proyectos-de-inversi%C3%B3n-del-gobierno-regional-de-lima

    List of investment projects of the Regional Government of Lima. This dataset is characterized by: Unique Investment Code, Sector to which the investment belongs, Multiannual Investment Programming Office to which the investment belongs, Responsible Formulation Unit, Responsible Investment Execution Unit, Geographic Location Code, Department in which it is find the investment, Province in which the investment is located, Function to which the investment belongs, Type of the Investment, Name of the Investment, Updated Cost of the investment, Accumulated Earnings as of 2021 of the investment, Modified Institutional Budget of the investment, Scheduled amount for the year 2022 of the investment, Scheduled amount for the year 2023 of the Investment, Scheduled amount for the year 2024 of the investment, Scheduled amount for the year 2025 of the investment, Cut-off date of the information.


    #### Type of Analysis can be done :    
    `Sectoral Analysis, Geographical Analysis, Financial Analysis, Multiannual Planning, Function and Type of Investment`    

8) **San Juan de Miraflores District Municipality**
https://www.datosabiertos.gob.pe/group/municipalidad-distrital-san-juan-de-miraflores 

    8.1) Income Collected from the District Municipality of San Juan de Miraflores – [MDSJM]
    https://www.datosabiertos.gob.pe/dataset/ingresos-recaudados-de-la-municipalidad-distrital-de-san-juan-de-miraflores-%E2%80%93-mdsjm

    Data on income from Tax Collection (tax and services) corresponding to the year 2022.

    This dataset is characterized by:
    Department where the Payment was made
    Province where the Payment was made
    District where the Payment was made
    Geographic location code where the payment was made
    Date of payment
    Id that identifies the taxpayer (anonymized)
    Person to whom payment is assigned (anonymized)
    Payment month
    Payment concept
    Years of accounts paid. Expressed in Peruvian Sol currency.
    Total paid per row. Expressed in Peruvian Sol currency.    

    #### Type of Analysis can be done :
    `Geographic Analysis and Contribution, Temporal Analysis and Patterns, Payment Concept Analysis, Financial Overview and Health, Revenue Sources`

9) **San Miguel District Municipality**
https://www.datosabiertos.gob.pe/group/municipalidad-distrital-de-san-miguel

    9.1) DEBTS FOR TAXES [DISTRITAL MUNICIPALITY OF SAN MIGUEL]
    https://www.datosabiertos.gob.pe/dataset/deudas-por-tributos-municipalidad-distrital-de-san-miguel

    Consultation of debts for taxes, contributions, rates, excise taxes or fines of a municipal nature by those administered [2020-2023].

    9.2) PREDIOS_VALUOS [DISTRITAL MUNICIPALITY OF SAN MIGUEL]
    https://www.datosabiertos.gob.pe/dataset/prediosvaluos-municipalidad-distrital-de-san-miguel

    Consultation of properties registered by the District Municipality of San Miguel [2021-2023]    

    #### Type of Analysis can be done :
    - Debts for Taxes \
    `Debt Distribution and Trends, Taxpayer Analysis and Behaviour, Temporal Analysis, Financial Health`
    - Predios Valuos \
    `Property Valuation, Tax Assessment, Urban Planning, Community Impact`

10) **Ministry of Labor and Employment Promotion**
https://www.datosabiertos.gob.pe/group/ministerio-de-trabajo-y-promoci%C3%B3n-del-empleo
    
    10.1) Monthly average of Private sector Workers by Pension Scheme, according to districts - [Ministry of Labor and Employment Promotion - MTPE]
    https://www.datosabiertos.gob.pe/dataset/promedio-mensual-de-trabajadores-del-sector-privado-por-r%C3%A9gimen-de-pensiones-seg%C3%BAn-distritos

    Monthly average of Private Sector Workers by Pension Regime, according to districts in 2021. This dataset presents: District ubigeo code, Type of Pension Regime where the worker is located.   

    #### Type of Analysis can be done :
    `Pension Scheme Distribution, Regional Disparities, Temporal Analysis, Labor Market Dynamics`

11) **Ministry of Production** https://www.datosabiertos.gob.pe/group/ministerio-de-la-producci%C3%B3n

    11.1) Directory of Large Companies in the Manufacturing sector [Ministry of Production - PRODUCE]
    https://www.datosabiertos.gob.pe/dataset/directorio-de-grandes-empresas-del-sector-manufactura-ministerio-de-la-producci%C3%B3n-produce

    Directory of large companies in the Manufacturing sector of Peru, Contains the companies that make the greatest contribution to the Manufacturing GDP.
    All the companies on this list are accredited by the National Superintendence of Customs and Tax Administration (SUNAT), that is, they have a RUC number.
    This dataset is characterized by: RUC, Ubigeo, CIIU    

    11.2) Directory of MSME Companies by productive sector [Ministry of Production - PRODUCE]
    https://www.datosabiertos.gob.pe/dataset/directorio-de-empresas-mipyme-por-sector-productivo-ministerio-de-la-producci%C3%B3n-produce

    Directory of MSME companies in Peru, categorized by productive sector (Commerce, Manufacturing, Services)
    All the companies on this list are accredited by the National Superintendence of Customs and Tax Administration (SUNAT), that is, they have a RUC number.
    This dataset is characterized by: Sector, Ubigeo, CIIU    

    11.3) Collective Interest and Benefit Companies (BIC) - [Ministry of Production - PRODUCE]
    https://www.datosabiertos.gob.pe/dataset/sociedades-de-beneficio-e-inter%C3%A9s-colectivo-bic-ministerio-de-producci%C3%B3n-produce

    List of Benefit and Collective Interest Companies (BIC companies), registered in the National Superintendence of Public Registries (SUNARP) and in the SUNAT of Peru.
    The dataset includes a link to the annual report of each company in PDF format, and the date on which said report was published.
    This dataset is published to comply with the Fifth Final Complementary Provision of DS No. 004-2021-PRODUCE of the Ministry of Production. 


    #### Type of Analysis can be done :
    - Directory of Large Companies in the Manufacturing Sector \
    `Sectoral Contribution, Contribution to GDP,  Geographical Distribution, Economic Impact, Industry Trends, Regional Development`
    - Directory of MSME Companies by Productive Sector \
    `Sectoral Distribution, Geographical Representation, Economic Impact, Regional Development`
    - Collective Interest and Benefit Companies (BIC) \
    `Legal Structure, Geographical Representation, Annual Reports, Social Impact`
